nro id : se não tiver length 16 adicionar 0 na frente

instituição
- hospital : não é de ensino, tem q marcar q não é de academia
- mesma instituição, registros e tempos diferentes  =>  tem q agrupar
similaridade de string -> função cosseno de PLN (ngramas)



1) pegar o nome de todas as instituições do brasil (usar a lista do MEC)


- fazer a tabela de frequência /
eliminar oq tem pouca frequência
- se for secretaria municipal algo assim e aparecer poucas vezes => só rotular como servidor público
- se estiver so EACH (sem ter USP no nome), categorizar como 'outros'
- lista de universidades
    - tabela do ROR
    - tabela do MEC

In [ ]:
from google.colab import drive
import os

import pandas as pd
import numpy as np

drive.mount('/content/gdrive')
os.chdir('/content/gdrive/My Drive/USP/2024_2º/MQA/')

pd.set_option('display.max_rows', 10)
pd.set_option('display.max_columns', None)

## READ_CSV

In [ ]:
df = pd.read_csv('atuacoes_2.csv', encoding='UTF-8', nrows=50000) #, dtype=str, nrows=1000)

df['ANO-FIM'] = df['ANO-FIM'].fillna(2024)
df['FLAG-DEDICACAO-EXCLUSIVA'] = df['FLAG-DEDICACAO-EXCLUSIVA'].replace('NAO', False).replace('SIM', True)

df = df.astype({
    'NRO-ID-CNPQ': int,
    'CODIGO-INSTITUICAO': str,
    'NOME-INSTITUICAO': str,
    'ANO-INICIO': int,
    'ANO-FIM': int,
    'ENQUADRAMENTO-FUNCIONAL': str,
    'TIPO-DE-VINCULO': str,
    'OUTRAS-INFORMACOES': str,
    'CARGA-HORARIA-SEMANAL': float,
    'FLAG-DEDICACAO-EXCLUSIVA': bool
})

df = df.fillna(' ')
df2 = df.copy()

df

## TRATAMENTO

In [ ]:
df2 = df2.apply(lambda x: x.astype(str).str.upper())

df2['NOME-INSTITUICAO'] = df2['NOME-INSTITUICAO'].replace('&AMP;', ' ', regex=True)
df2['NOME-INSTITUICAO'] = df2['NOME-INSTITUICAO'].str.replace('[^\w\s]', ' ', regex=True)
df2['NOME-INSTITUICAO'] = df2['NOME-INSTITUICAO'].str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8')

df2

In [ ]:
df2['ENQUADRAMENTO-FUNCIONAL'].value_counts()
# apenas levar em consideracao os LIVRE, PROFESSOR_VISITANTE ?
# la embaixo, secretaria~~ ta atrapalhando

In [ ]:
df2['NOME-INSTITUICAO'].value_counts()

In [ ]:
# drop frequency less than 50
df2 = df2[df2.groupby('NOME-INSTITUICAO')['NOME-INSTITUICAO'].transform('count').ge(50)]
df2['NOME-INSTITUICAO'].value_counts()

In [ ]:
# LISTA DE UNIVERSIDADES
# https://dadosabertos.mec.gov.br/indicadores-sobre-ensino-superior/item/181-instituicoes-de-educacao-superior-do-brasil
# https://dadosabertos.mec.gov.br/images/conteudo/Ind-ensino-superior/2022/PDA_Lista_Instituicoes_Ensino_Superior_do_Brasil_EMEC.csv

'''
criar uma categoria
1 : educacao
0 : outros
'''

df_uni = pd.read_csv('sup.csv', encoding='UTF-8')
df_uni_filtered = df_uni[['NOME_DA_IES', 'SIGLA']]
df_uni_filtered = df_uni_filtered.fillna('')

df_uni_filtered = df_uni_filtered.apply(lambda x: x.astype(str).str.upper())
df_uni_filtered['NOME_DA_IES'] = df_uni_filtered['NOME_DA_IES'].replace('&AMP;', ' ', regex=True)
df_uni_filtered['NOME_DA_IES'] = df_uni_filtered['NOME_DA_IES'].str.replace('[^\w\s]', ' ', regex=True)
df_uni_filtered['NOME_DA_IES'] = df_uni_filtered['NOME_DA_IES'].str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8')

# df_uni_filtered['NOME_E_SIGLA'] = df_uni_filtered['NOME_DA_IES'] + ' ' + df_uni_filtered['SIGLA']
df_uni_filtered

## NLP

### Cosine

In [ ]:
# SIMILARIDADE POR NOME COMPLETO SEM SIGLA

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# Vetorização TF-IDF
vectorizer = TfidfVectorizer().fit(df_uni_filtered['NOME_DA_IES'].values)
tfidf_matrix_1 = vectorizer.transform(df_uni_filtered['NOME_DA_IES'].values)
tfidf_matrix_2 = vectorizer.transform(df2['NOME-INSTITUICAO'].values)

# Calcular a similaridade de cosseno
similaridades = cosine_similarity(tfidf_matrix_2, tfidf_matrix_1)

# Para cada universidade no df2, encontre o índice do nome mais similar no df1
df2['indice_mais_similar'] = similaridades.argmax(axis=1)
df2['similaridade_maxima'] = similaridades.max(axis=1)

threshold = 0.75

# Substituir os nomes no df2 pelo nome correspondente do df1
df2['nome_corrigido'] = df2.apply(
    lambda row: df_uni_filtered.iloc[row['indice_mais_similar']]['NOME_DA_IES']
    if row['similaridade_maxima'] >= threshold
    else row['NOME-INSTITUICAO'], axis=1
)

# Exibir o resultado final
#df2[['NOME-INSTITUICAO', 'nome_corrigido']]
#df2.loc[df2['NOME-INSTITUICAO'] != df2['nome_corrigido']]

In [ ]:
pd.set_option('display.max_rows', None)
print(df2[['NOME-INSTITUICAO', 'nome_corrigido']].loc[df2['NOME-INSTITUICAO'] != df2['nome_corrigido']])
pd.set_option('display.max_rows', 10)

In [ ]:
# SIMILARIDADE POR SIGLA
# NAO FUNCIONA, VER SE CONTEM

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# Vetorização TF-IDF
vectorizer = TfidfVectorizer().fit(df_uni_filtered['SIGLA'].values)
tfidf_matrix_1 = vectorizer.transform(df_uni_filtered['SIGLA'].values)
tfidf_matrix_2 = vectorizer.transform(df2['NOME-INSTITUICAO'].values)

# Calcular a similaridade de cosseno
similaridades = cosine_similarity(tfidf_matrix_2, tfidf_matrix_1)

# Para cada universidade no df2, encontre o índice do nome mais similar no df1
df2['indice_mais_similar_sigla'] = similaridades.argmax(axis=1)

# Substituir os nomes no df2 pelo nome correspondente do df1
df2['nome_corrigido_sigla'] = df2['indice_mais_similar_sigla'].apply(lambda idx: df_uni_filtered.iloc[idx]['SIGLA'])

# Exibir o resultado final
print(df2[['NOME-INSTITUICAO', 'nome_corrigido_sigla']])

# SE TROCOU, ATRIBUIR A COLUNA 'ROTULO' COMO EDUCACAO
# SE NÃO, ATRIBUIR A COLUNA COMO OUTROS

### N-gram


In [ ]:
vectorizer_ngram = TfidfVectorizer(analyzer='char', ngram_range=(2, 4)).fit(df_uni_filtered['NOME_DA_IES'].values)
tfidf_matrix_1_ngram = vectorizer.transform(df_uni_filtered['NOME_DA_IES'].values)
tfidf_matrix_2_ngram = vectorizer.transform(df2['NOME-INSTITUICAO'].values)

similaridades_ngram = cosine_similarity(tfidf_matrix_2_ngram, tfidf_matrix_1_ngram)

# Para cada universidade no df2, encontre o índice do nome mais similar no df1
df2['indice_mais_similar_ngram'] = similaridades_ngram.argmax(axis=1)
df2['similaridade_maxima_ngram'] = similaridades_ngram.max(axis=1)

threshold = 0.8

# Substituir os nomes no df2 pelo nome correspondente do df1
df2['nome_corrigido_ngram'] = df2.apply(
    lambda row: df_uni_filtered.iloc[row['indice_mais_similar_ngram']]['NOME_DA_IES']
    if row['similaridade_maxima_ngram'] >= threshold
    else row['NOME-INSTITUICAO'], axis=1
)

# Exibir o resultado final
#df2[['NOME-INSTITUICAO', 'nome_corrigido']]
#df2.loc[df2['NOME-INSTITUICAO'] != df2['nome_corrigido']]

In [ ]:
pd.set_option('display.max_rows', 10)
print(df2[['NOME-INSTITUICAO', 'nome_corrigido_ngram']].loc[df2['NOME-INSTITUICAO'] != df2['nome_corrigido_ngram']])

### Word Embedding
- muito lentooo
- 50k linhas : 3min

In [ ]:
!python -m spacy download pt_core_news_sm
import spacy

# AJUSTAR O CODIGO PQ NAO TROCA A COLUNA NEM NADA

nlp = spacy.load("pt_core_news_sm")  # Carrega um modelo pré-treinado
df2['spacy_similarity'] = df2.apply(
    lambda row: nlp(row['NOME-INSTITUICAO']).similarity(nlp(df_uni_filtered['NOME_DA_IES'][row['indice_mais_similar']])),
    axis=1
)

df2

### Word2Vect

# CAIO

## MERGE SAME UNI

In [ ]:
# merge ANO-INICIO and ANO-FIM with same CODIGO-INSTITUICAO and NRO-ID-CNPQ

df_merged = df.groupby(['NRO-ID-CNPQ', 'CODIGO-INSTITUICAO']).agg(
    earliest_date1=('ANO-INICIO', 'min'),
    latest_date2=('ANO-FIM', 'max')
).reset_index()


# df.loc[df['NRO-ID-CNPQ'] == 3300778291054405]

In [ ]:
# merge column NOME-INSTITUICAO with df_merged

df_drop_duplicates = df.drop_duplicates(subset=['NRO-ID-CNPQ', 'CODIGO-INSTITUICAO'])
df_inst_merged = pd.merge(df_merged, df_drop_duplicates, on=['CODIGO-INSTITUICAO', 'NRO-ID-CNPQ'], how='left')
df_inst_merged['NOME-INSTITUICAO'] = df_inst_merged['NOME-INSTITUICAO'].str.upper()

df_inst_merged = df_inst_merged[['NRO-ID-CNPQ', 'CODIGO-INSTITUICAO', 'earliest_date1', 'latest_date2', 'NOME-INSTITUICAO']]
df_inst_merged

## NLP

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# The target word for filtering (single word)
target_word = "UNIVERSIDADE DE SÃO PAULO"

# Step 1: Turn phrases into TF-IDF vectors
vectorizer = TfidfVectorizer()
phrase_vectors = vectorizer.fit_transform(df_inst_merged['NOME-INSTITUICAO'])

# Step 2: Convert the target word into a vector
# We will use the same vectorizer to transform the target word
target_vector = vectorizer.transform([target_word])

# Step 3: Compute cosine similarity between the target word and all phrases
similarity_scores = cosine_similarity(target_vector, phrase_vectors).flatten()

df_inst_merged['PREDICTION'] = similarity_scores

# Step 4: Filter phrases based on a similarity threshold (e.g., > 0.1)
threshold = 0.7
filtered_phrases = df_inst_merged[similarity_scores > threshold]

# Display the phrases that have similarity above the threshold
#print("Filtered Phrases based on Cosine Similarity with the word 'sky':")
#print(filtered_phrases)
df_inst_merged

In [ ]:
df_inst_merged.to_csv("atuacoes_2_filtrado.csv", encoding='utf8')